In [ ]:
import os
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D
from keras import Sequential, layers
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
from PIL import Image

# **Collect data and Pre-processing**

In [ ]:
path = "drive/MyDrive/emotion_detection/eINTERFACE_2021_Image/"
train_dir = "train_data"
test_dir = "test_data"
list_folder = []

for name in os.listdir(path + train_dir):
  list_folder.append(name)
  print(name)

Fear
Surprise
Happiness
Sadness
Disgust
Anger


In [ ]:
def collect_data_from(path, data_dir):
  data_image, data_label = [], []
  label = 0
  for folder in list_folder:
    for link_image in os.listdir(path + data_dir + '/' + folder):
      image = cv2.imread(path + data_dir + '/' + folder + '/' + link_image, cv2.IMREAD_GRAYSCALE)
      resized_image = cv2.resize(image, (128, 128))
      # image = mpimg.imread(path + data_dir + '/' + folder + '/' + link_image)
      data_image.append(resized_image)
      data_label.append(label)
    label += 1
  return data_image, data_label

train_image, train_label = collect_data_from(path, train_dir)
test_image, test_label = collect_data_from(path, test_dir)

In [ ]:
train_image = np.array(train_image)
test_image = np.array(test_image)
train_label = np.array(train_label)
test_label = np.array(test_label)
dict_label = {1: 'Fear', 2: 'Surprise', 3: 'Happiness', 4: 'Sadness', 5: 'Disgust', 6: 'Anger'}
'''
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(path + train_dir,
                                                 target_size = (128, 128),
                                                 batch_size = 64,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(path + test_dir,
                                            target_size = (128, 128),
                                            batch_size = 64,
                                            class_mode = 'categorical')
'''

"\nfrom keras.preprocessing.image import ImageDataGenerator\n\ntrain_datagen = ImageDataGenerator(rescale = 1./255,\n                                   shear_range = 0.2,\n                                   zoom_range = 0.2,\n                                   horizontal_flip = True)\n\ntest_datagen = ImageDataGenerator(rescale = 1./255)\n\ntraining_set = train_datagen.flow_from_directory(path + train_dir,\n                                                 target_size = (128, 128),\n                                                 batch_size = 64,\n                                                 class_mode = 'categorical')\n\ntest_set = test_datagen.flow_from_directory(path + test_dir,\n                                            target_size = (128, 128),\n                                            batch_size = 64,\n                                            class_mode = 'categorical')\n"

# **Building CNN structure to train data**

In [ ]:
# Build model

model = Sequential()

model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation = 'relu', input_shape = (128, 128, 1)))
model.add(layers.BatchNormalization())
model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu'))
model.add(layers.BatchNormalization())
model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.3))

model.add(layers.Flatten())
model.add(Dense(units = 128, activation = 'relu'))
model.add(layers.BatchNormalization())
model.add(Dense(units = 6, activation = 'softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 16)      160       
                                                                 
 batch_normalization (BatchN  (None, 126, 126, 16)     64        
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 124, 124, 32)      4640      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 62, 62, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 60, 60, 32)       1

In [ ]:
model.compile(optimizer = 'SGD', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(train_image, train_label, epochs = 5, validation_data = (test_image, test_label))

Epoch 1/5
360/360 [==============================] - 715s 2s/step - loss: 1.3642 - accuracy: 0.4748 - val_loss: 2.4598 - val_accuracy: 0.3825
Epoch 2/5
360/360 [==============================] - 704s 2s/step - loss: 0.8398 - accuracy: 0.6871 - val_loss: 0.9443 - val_accuracy: 0.6321
Epoch 3/5
360/360 [==============================] - 711s 2s/step - loss: 0.6348 - accuracy: 0.7708 - val_loss: 1.0598 - val_accuracy: 0.6495
Epoch 4/5
360/360 [==============================] - 704s 2s/step - loss: 0.5013 - accuracy: 0.8228 - val_loss: 1.3877 - val_accuracy: 0.6210
Epoch 5/5
360/360 [==============================] - 704s 2s/step - loss: 0.4121 - accuracy: 0.8550 - val_loss: 1.1214 - val_accuracy: 0.6161


In [ ]:
model.evaluate(test_image, test_label)

45/45 [==============================] - 27s 605ms/step - loss: 1.1214 - accuracy: 0.6161


[1.1213911771774292, 0.6161335110664368]

In [ ]:
# show image function
def show_image(data_image):
    prediction_prob = model.predict(data_image)
    label_max = np.argmax(prediction_prob)

    image = np.squeeze(data_image, axis = 0)
    plt.imshow(image)
    plt.xlabel("Predict emotion: " + dict_label[label_max + 1])

def resize(image_path):
  # read image and convert to gray scale
  original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

  # resize the image to 128 x 128 pixels
  resized_image = cv2.resize(original_image, (128, 128))
  resized_image = np.expand_dims(resized_image, axis = 0)
  return resized_image

image_1 = resize("dang.jpg")